In [1]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.base import RunnableParallel

from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

In [2]:
llm = GoogleGenerativeAI(model="gemini-2.5-flash-lite")
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant -- Gemini 2.5 flash lite."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [3]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

### **Simple Buffer Memory**

In [4]:
chain = prompt_template | llm

chain_with_buffer_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_variables="input",
    # output_variables=None,
    history_messages_key="chat_history",
)

In [5]:
chain_with_buffer_memory.invoke(
    {"input": "My name is John."},
    config={"configurable": {"session_id": "simple_memory_llm"}}
)

"Hi John! It's nice to meet you. How can I help you today?"

In [6]:
chain_with_buffer_memory.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "simple_memory_llm"}}
)

'Your name is John.'

In [7]:
print(get_session_history("simple_memory_llm"))

Human: My name is John.
AI: Hi John! It's nice to meet you. How can I help you today?
Human: What is my name?
AI: Your name is John.


### **k buffer window**

In [8]:
from langchain_core.messages.utils import trim_messages

sample_messages = [
    HumanMessage("Hi, I'm looking for a book on quantum physics."),
    AIMessage("Hello! We have several options. Are you looking for something for beginners or a more advanced text?"),
    HumanMessage("I'm a beginner, so something introductory would be great."),
    AIMessage("I recommend 'Quantum Enigma' by Bruce Rosenblum. It explains complex ideas in an easy-to-understand way."),
    HumanMessage("That sounds perfect. Do you have it in stock?"),
    AIMessage("Yes, we do. I can have a copy held for you at the front desk."),
    HumanMessage("Thank you so much!"),
    AIMessage("You're welcome! Let me know if you need anything else.")
]

In [9]:
# keep k last tokens: `token_counter=BaseLanguageModel`
trim_messages(
    sample_messages,
    max_tokens=50,
    strategy="last",
    token_counter=llm,
    # start_on="human",
    include_system=False,
    allow_partial=False
)

[AIMessage(content='Yes, we do. I can have a copy held for you at the front desk.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thank you so much!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="You're welcome! Let me know if you need anything else.", additional_kwargs={}, response_metadata={})]

In [10]:
# keep k last messages: `token_counter=len`
trim_messages(
    sample_messages,
    max_tokens=4,
    strategy="last",
    token_counter=len,
    # start_on="human",
    include_system=False,
    allow_partial=False
)

[HumanMessage(content='That sounds perfect. Do you have it in stock?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes, we do. I can have a copy held for you at the front desk.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thank you so much!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="You're welcome! Let me know if you need anything else.", additional_kwargs={}, response_metadata={})]

In [ ]:
def trim_chat_history(x):
    trimmed_messages = trim_messages(
        x['chat_history'],
        max_tokens=2,
        strategy="last",
        token_counter=len,
        include_system=False,
        allow_partial=False
    )
    return trimmed_messages

# {"input": ..., "chat_history": messeges]}
# --> {"input": ..., "chat_history": trimmed_messeges}
chain = RunnablePassthrough.assign(chat_history=trim_chat_history) | prompt_template | llm

chain_with_buffer_window_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history, 
    input_variables="input",
    # output_variables=None,
    history_messages_key="chat_history",
)

In [12]:
chain_with_buffer_window_memory.invoke(
    {"input": "My name is John."},
    config={"configurable": {"session_id": "buffer_window_llm"}}
)

"Hello John! It's nice to meet you. How can I help you today?"

In [13]:
chain_with_buffer_window_memory.invoke(
    {"input": "What is your name?"},
    config={"configurable": {"session_id": "buffer_window_llm"}}
)

'I do not have a name. I am a large language model, trained by Google.'

In [14]:
# LLM 不會知道 John，因為修剪到最多兩條記憶
chain_with_buffer_window_memory.invoke(
    {"input": "What is my name:"},
    config={"configurable": {"session_id": "buffer_window_llm"}}
)

'I do not have access to your personal information, so I cannot tell you your name.'

In [15]:
print(get_session_history("buffer_window_llm"))

Human: My name is John.
AI: Hello John! It's nice to meet you. How can I help you today?
Human: What is your name?
AI: I do not have a name. I am a large language model, trained by Google.
Human: What is my name:
AI: I do not have access to your personal information, so I cannot tell you your name.


### **Summary Memory**

In [ ]:
def debug(x):
    print(f"Summary: {x['summary']}", end=f"\n\n{'-' * 20}\n\n")
    return x

summarizer = GoogleGenerativeAI(model="gemini-2.5-flash-lite")
summary_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please summarize the following conversation history concisely. If the list is empty, just return nothing."),
        MessagesPlaceholder(variable_name="chat_history"), 
    ]
)
main_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant. The conversation so far is summarized as follows: {summary}. If the list is empty, it means it's the start of the conversation."),
        ("human", "{input}"),
    ]
)

summarization_chain = RunnablePassthrough.assign(chat_history=lambda x: x["chat_history"]) | summary_prompt | summarizer

runnable = RunnableParallel(
    input=RunnablePassthrough(func=lambda x: x['input']),
    summary=summarization_chain
)

# {"input": ..., "chat_history": messeges]}
# --> {"input": ..., "summary": summarized_messeges]}
chain = runnable | debug | main_prompt | llm

chain_with_summary_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history, 
    input_variables="input",
    # output_variables=None,
    history_messages_key="chat_history",
)

In [17]:
print(chain_with_summary_memory.invoke(
    {"input": r"Evaluate the derivative: \[ \frac{d}{dx} \sin(\ln(x^2)))\]"},
    config={"configurable": {"session_id": "summary_llm"}}
))

Summary: The conversation history is empty.

--------------------

To evaluate the derivative of $\sin(\ln(x^2))$, we need to use the chain rule. The chain rule states that if we have a composite function $f(g(x))$, its derivative is $f'(g(x)) \cdot g'(x)$.

In this case, our outer function is $f(u) = \sin(u)$ and our inner function is $g(x) = \ln(x^2)$.

First, let's find the derivative of the outer function:
$f'(u) = \frac{d}{du} \sin(u) = \cos(u)$

Next, let's find the derivative of the inner function, $g(x) = \ln(x^2)$. We'll need to use the chain rule again for this part, as the argument of the logarithm is itself a function of $x$.
Let $h(x) = x^2$. Then $g(x) = \ln(h(x))$.
The derivative of $\ln(u)$ is $\frac{1}{u}$.
The derivative of $h(x) = x^2$ is $h'(x) = 2x$.

So, the derivative of $g(x) = \ln(x^2)$ is:
$g'(x) = \frac{d}{dx} \ln(x^2) = \frac{1}{x^2} \cdot \frac{d}{dx}(x^2) = \frac{1}{x^2} \cdot 2x = \frac{2x}{x^2} = \frac{2}{x}$.

Now, we can apply the chain rule to the ori

In [18]:
print(chain_with_summary_memory.invoke(
    {"input": "What did i ask you?"},
    config={"configurable": {"session_id": "summary_llm"}}
))

Summary: The user asked to evaluate the derivative of $\sin(\ln(x^2))$. The AI used the chain rule twice to find the derivative, first for the outer sine function and then for the inner logarithm function. The final result was $\frac{2 \cos(\ln(x^2))}{x}$.

--------------------

You asked me to evaluate the derivative of $\sin(\ln(x^2))$.
